In [139]:
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import numpy as np
import random

In [77]:
length_sums = []
lefts = []
rights = []
left_values = []
right_values = []
common_values = []

with open('lines.txt', 'r') as file:
    count = int(file.readline())
    for i in range(count):
        tokens = file.readline().split()
        left_len = int(tokens[0])
        right_len = int(tokens[1])
        left_value = float(tokens[2])
        right_value = float(tokens[3])
        common_value = float(tokens[4])
        lefts.append(file.read(left_len))
        rights.append(file.read(right_len))
        length_sums.append(left_len + right_len)
        left_values.append(left_value)
        right_values.append(right_value)
        common_values.append(common_value)

In [25]:
%matplotlib

Using matplotlib backend: TkAgg


In [46]:
print(len(ious), len(length_sums))
plt.scatter(ious, length_sums, s=1)
plt.show()

25000 25000


In [200]:
def extract_piece_set(string, k):
    pieces = set()
    occurences = {}
    for i in range(len(string) - k + 1):
        piece = string[i : i + k]
        if piece not in occurences:
            occurences[piece] = 0
        #pieces.add(piece)
        pieces.add((occurences[piece], piece))
        occurences[piece] += 1
    return pieces

def est_exact(left_set, right_set):
    intersection = 0
    for left_piece in left_set:
        if left_piece in right_set:
            intersection += 1
    return intersection

def est_with_replacement(left_set, right_set, samples = 5):
    hits = 0
    for round in range(samples):
        salt = random.randrange(0, 2 ** 32)
        left_max = -2 ** 64
        for piece in left_set:
            left_max = max(left_max, hash((salt, piece)))
        right_max = -2 ** 64
        for piece in right_set:
            right_max = max(right_max, hash((salt, piece)))
        if left_max == right_max:
            hits += 1
    iou = hits / samples
    return (len(left_set) + len(right_set)) * iou / (1 + iou)



def est_bitvector_with_replacement(left_set, right_set, samples, vec_len, correct=True):
    left_bitvector = np.zeros(vec_len, dtype=bool)
    left_bits = 0
    
    right_bitvector = np.zeros(vec_len, dtype=bool)
    right_bits = 0
    
    salts = [random.randrange(0, 2 ** 32) for _ in range(samples)]
    
    for salt in salts:
        left_max = -2 ** 64
        for piece in left_set:
            left_max = max(left_max, hash((salt, piece)))
        if not left_bitvector[left_max % vec_len]:
            left_bitvector[left_max % vec_len] = True
            left_bits += 1
    
    hits = 0
    
    for salt in salts:
        right_max = -2 ** 64
        for piece in right_set:
            right_max = max(right_max, hash((salt, piece)))
        if not right_bitvector[right_max % vec_len]:
            right_bitvector[right_max % vec_len] = True
            right_bits += 1
            if left_bitvector[right_max % vec_len]:
                hits += 1
    if left_bits + right_bits == 0:
        return 0
    
    corrected_hits = max((hits - (left_bits * right_bits) / vec_len), 0) / (1 - max(left_bits, right_bits) / vec_len)
    
    return (corrected_hits if correct else hits) / (left_bits + right_bits) * (len(left_set) + len(right_set))


    
def est_without_replacement(left_set, right_set, samples = 5):
    left_hashes = list(map(hash, left_set))
    left_hashes.sort()
    left_hashes = left_hashes[-samples:]
    right_hashes = list(map(hash, right_set))
    right_hashes.sort()
    right_hashes = right_hashes[-samples:]
    hits = 0
    for left in left_hashes:
        if left in right_hashes:
            hits += 1
    iou = hits / samples
    return (len(left_set) + len(right_set)) * iou / (1 + iou)


def est_bitvector_without_replacement(left_set, right_set, samples, vec_len, correct=True):
    left_hashes = list(map(hash, left_set))
    left_hashes.sort()
    left_hashes = left_hashes[-samples:]
    right_hashes = list(map(hash, right_set))
    right_hashes.sort()
    right_hashes = right_hashes[-samples:]
    
    left_bitvector = np.zeros(vec_len, dtype=bool)
    left_bits = 0
    for h in left_hashes:
        if not left_bitvector[h % vec_len]:
            left_bitvector[h % vec_len] = True
            left_bits += 1
    
    hits = 0
    right_bitvector = np.zeros(vec_len, dtype=bool)
    right_bits = 0
    for h in right_hashes:
        if not right_bitvector[h % vec_len]:
            right_bitvector[h % vec_len] = True
            right_bits += 1
            if left_bitvector[h % vec_len]:
                hits += 1
    if left_bits + right_bits == 0:
        return 0
    
    corrected_hits = max((hits - (left_bits * right_bits) / vec_len), 0) / (1 - max(left_bits, right_bits) / vec_len)
    
    return (corrected_hits if correct else hits) / (left_bits + right_bits) * (len(left_set) + len(right_set))
    
def predict_score(left, right, left_value, right_value, k, intersection_estimator):
    left_pieces = extract_piece_set(left, k)
    right_pieces = extract_piece_set(right, k)
    intersection = intersection_estimator(left_pieces, right_pieces)
        
    total_pieces = len(left_pieces) + len(right_pieces)
    if total_pieces == 0:
        return 0.0
    return intersection / total_pieces * (left_value + right_value)

In [206]:
for samples in range(26, 27, 2):
    estimator = lambda left, right: est_bitvector_with_replacement(left, right, samples, 128, True)
    for k in [3]:
        predicted = [predict_score(lefts[i], rights[i], left_values[i], right_values[i], k, estimator) for i in range(len(lefts))]
        corr, _ = pearsonr(common_values, predicted)
        print(k, corr)
        plt.scatter(common_values, predicted, s=1)
plt.show()

3 0.9401979958176483


In [198]:
correlations = []
for samples in range(1, 256, 8):
    estimator = lambda left, right: est_bitvector(left, right, samples, 256, False)
    for k in [3]:
        predicted = [predict_score(lefts[i], rights[i], left_values[i], right_values[i], k, estimator) for i in range(len(lefts))]
        corr, _ = pearsonr(common_values, predicted)
        print(k, corr)
        correlations.append(corr)
        plt.scatter(common_values, predicted, s=1)
plt.show()

3 0.689933632644842
3 0.9472622279959426
3 0.9646222942806633
3 0.9771308506986505
3 0.9780875633664778
3 0.9812675715198768
3 0.9847183811518085
3 0.9841089592719765
3 0.9829665994616549
3 0.9814526701144386
3 0.9802103454051657
3 0.9790197700383816
3 0.9769738950535726
3 0.9754771904009086
3 0.9752809511934463
3 0.9750336135552756
3 0.9749866683192061
3 0.974927360993288
3 0.9748586830545237
3 0.9748362457211789
3 0.9747889300158157
3 0.9747877941354356
3 0.9747572820892314
3 0.9747195099738948
3 0.9747196137385217
3 0.9747168130007811
3 0.9747207750618045
3 0.9747207750618045
3 0.9747207750618045
3 0.9747207750618045
3 0.9747207750618045
3 0.9747207750618045


In [199]:
plt.plot(correlations)

2.7184177550101496